## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-07-13-14-27-12 +0000,nypost,Karen Read lands lucrative book deal while wit...,https://nypost.com/2025/07/13/us-news/karen-re...
1,2025-07-13-14-20-54 +0000,nypost,Kash Patel insists Epstein ‘conspiracy theorie...,https://nypost.com/2025/07/13/us-news/kash-pat...
2,2025-07-13-14-20-15 +0000,bbc,"Electric cars will be cheaper to buy, pledges ...",https://www.bbc.com/news/articles/cg5z4nlned0o
3,2025-07-13-13-44-03 +0000,nypost,"Gov. Newsom, protesters target JD Vance with h...",https://nypost.com/2025/07/13/us-news/gov-news...
4,2025-07-13-13-40-45 +0000,bbc,Gisèle Pelicot given France's highest award,https://www.bbc.com/news/articles/cr5v1p4zel8o


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2308/2741963311.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
54,trump,11
48,killed,5
151,india,4
150,air,4
126,tariffs,3


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
50,2025-07-12-17-01-00 +0000,wsj,The U.S. will charge a 30% tariff on the Europ...,https://www.wsj.com/economy/trade/trump-threat...,33
32,2025-07-12-23-56-51 +0000,nypost,Trump defends Pam Bondi hours after FBI boss K...,https://nypost.com/2025/07/12/us-news/trump-de...,30
16,2025-07-13-09-01-31 +0000,nyt,"To Sidestep Trump Tariffs, Asian Nations Seek ...",https://www.nytimes.com/2025/07/13/business/ec...,23
7,2025-07-13-12-58-08 +0000,nypost,President Trump reveals what he really wants h...,https://nypost.com/2025/07/13/us-news/presiden...,23
6,2025-07-13-13-30-43 +0000,nypost,Trump tax break for auto loans could save buye...,https://nypost.com/2025/07/13/us-news/trump-ta...,23


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
50,33,2025-07-12-17-01-00 +0000,wsj,The U.S. will charge a 30% tariff on the Europ...,https://www.wsj.com/economy/trade/trump-threat...
29,23,2025-07-13-01-00-00 +0000,wsj,Economists expect stronger growth and job crea...,https://www.wsj.com/economy/economists-see-low...
22,23,2025-07-13-04-42-56 +0000,nypost,DOJ drops charges against Utah surgeon accused...,https://nypost.com/2025/07/13/us-news/doj-drop...
47,22,2025-07-12-21-11-05 +0000,bbc,The mushroom killer was obsessed with true cri...,https://www.bbc.com/news/articles/c0m8glx2zleo
19,21,2025-07-13-07-16-33 +0000,nyt,"For Families of Air India Crash Victims, Repor...",https://www.nytimes.com/2025/07/13/world/asia/...
32,19,2025-07-12-23-56-51 +0000,nypost,Trump defends Pam Bondi hours after FBI boss K...,https://nypost.com/2025/07/12/us-news/trump-de...
20,19,2025-07-13-06-27-28 +0000,nypost,"Violent brute allegedly shot parents, teen sis...",https://nypost.com/2025/07/13/us-news/new-mexi...
25,18,2025-07-13-03-06-33 +0000,nypost,Knife-wielding robbery suspect on e-bike hit a...,https://nypost.com/2025/07/12/us-news/knife-wi...
26,18,2025-07-13-02-44-25 +0000,nypost,Border czar Tom Homan delivers fiery response ...,https://nypost.com/2025/07/12/us-news/border-c...
45,15,2025-07-12-21-56-09 +0000,nypost,Video appears to show woman plowing through IC...,https://nypost.com/2025/07/12/us-news/woman-dr...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
